# addition prediction problem & encoder-decoder lstm

in their 2015 [paper](https://arxiv.org/pdf/1410.4615.pdf "https://arxiv.org/pdf/1410.4615.pdf"), Wojciech Zaremba and Ilya Sutskever showed that LSTM encoder-decoder models were capable of calculating the output of small programs--adding together two numbers of up to nine digits in length.

even more impressive, the model was reading the character representations of the symbols and digits. there is nothing programatically to indicate to the model what operations are being represented--it learns this during training:

>*It is important to emphasize that the LSTM reads the entire input one character at a time and produces the output one character at a time. The characters are initially meaningless from the model’s
perspective;  for instance, the model does not know that “+” means addition or that
6
is followed
by
7
.  In fact, scrambling the input characters (e.g., replacing “a” with “q”, “b” with “w”, etc.,) has
no effect on the model’s ability to solve this problem*
>
> Wojciech Zaremba and Ilya Sutskever, Learning to Execute

as an example of how this works, the model might take in the sequence representing

12 + 4 = 16

represented in the following vectors:

`['1','2','+','5']`

`['1','7']`

the digits and symbols are just characters; they have no fucntional meaning. the model learns the relationships during training.


## sequence-to-sequence (seq2seq) model

there are a couple of key characteristics to note in this problem. first, the order matters--shuffling the order of the characters would make any relationship impossible to deduce.

second, the input and output can vary, making this problem more challenging than a one-to-one or many-to-one sequence prediction problem.

because the data is an ordered sequence of variable input and output length, this problem requires a many-to-many modeling approach, otherwise known as [__sequence to sequence__](https://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf "https://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf").

### padding

because sequence lengths can vary, we need to do some __padding__. padding consists of adding characters at the beginning, end (or both) of a sequence to make sure all the sequences in a training set are the same length. any character can be used; but it should be sufficiently different from the training data that the mahcine can figure out it isn't pertinent to the problem at hand--in other words, the padding shoulnd't add noise.

there are different methods for choosing what characters to use for padding and how. some are intuitive, and some are statistical. 

*for more information about libraries for padding:*

__tensorflow__

*tool:* `tf.pad`

*documentation:* https://www.tensorflow.org/api_docs/python/tf/pad

__keras__

*tool:* `pad_sequences`

*documentation:* https://keras.io/preprocessing/sequence/

__numpy__

*tool:* `numpy.pad`

*documentation:* https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.pad.html

### one hot encoding

in order to be machine readable, these characters need to somehow be encoded into numerical data.

in this case, [__one hot encoding__](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f "https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f") provides the answer.

we can treat each character as a category, and each input to the machine will come with a set of vectors defining it. it's interesting to note here that in a sense the dense matrices associated with one hot encoding mostly serve to tell a machine what categories a particular example *isn't* in; these matrices are mostly zeros.

for automatic one hot encoding, there are a number of libraries and packages available. these mostly work well when every potential category is represented in the data.


### generating data

the code below generates data perfectly prepared for this problem. when executed, it will

* __generate__ random pairs of numbers with their sums
* convert the __integers__ to __strings__
* __pad__ the strings on the left, using the space `' '` character
* __integer encode__ the sequences

and, finally

* __one hot encode__ the sequences
* __assign__ sequences to data structures (lists) so they're ready for a model

because this code does all these things manually, it's easy to see how many of these parameters can be changed to experiement with model performance!

the code to generate this data below is taken (with only a few modifications) from [jason brownlee's excellent course on LSTMs](https://machinelearningmastery.com/lstms-with-python/ "https://machinelearningmastery.com/lstms-with-python/") available at [www.machinelearningmastery.com](https://machinelearningmastery.com/lstms-with-python/)

In [215]:

from random import seed
from random import randint
from math import ceil
from math import log10

# generate lists of random integers and their sum

def random_sum_pairs(n_examples, n_numbers, largest):
    
    X, y = list(), list()
    
    for _ in range(n_examples):
        
        in_pattern = [randint(1,largest) for _ in range(n_numbers)]
        
        out_pattern = sum(in_pattern)
        
        X.append(in_pattern)
        
        y.append(out_pattern)
    
    return X, y

# convert data to strings

def to_string(X, y, n_numbers, largest):
    
    # calculate largest possible value
    
    max_length = int(n_numbers * ceil(log10(largest+1)) + n_numbers - 1)
    
    Xstr = list()
    
    for pattern in X:
        
        strp = '+'.join([str(n) for n in pattern])
        
        strp = ''.join([' ' for _ in range(max_length-len(strp))]) + strp
        
        Xstr.append(strp)
    
    max_length = int(ceil(log10(n_numbers * (largest+1))))
    
    ystr = list()
    
    for pattern in y:
        
        strp = str(pattern)
        
        strp = ''.join([' ' for _ in range(max_length-len(strp))]) + strp
        
        ystr.append(strp)
    
    return Xstr, ystr

# integer encode strings
# i've changed variable names here to prevent scope issues with multiple notebook runs
# and make it easier for me to read

def integer_encode(X, y, alphabet):
    
    char_to_int = dict((c, i) for i, c in enumerate(alphabet))
    
    X_int = list()
    
    for pattern in X:
        
        integer_encoded = [char_to_int[char] for char in pattern]
        
        X_int.append(integer_encoded)
    
    y_int = list()
    
    for pattern in y:
        
        integer_encoded = [char_to_int[char] for char in pattern]
        
        y_int.append(integer_encoded)
    
    return X_int, y_int

# one hot encode
# some names changed for scope & readability

def one_hot_encode(X, y, max_int):
    
    X_encoded = list()
    
    for seq in X:
        
        pattern = list()
            
        for index in seq:
                
            vector = [0 for _ in range(max_int)]
                
            vector[index] = 1
                
            pattern.append(vector)
        
        X_encoded.append(pattern)
    
    y_encoded = list()
    
    for seq in y:
        
        pattern = list()
        
        for index in seq:
            
            vector = [0 for _ in range(max_int)]
            
            vector[index] = 1
            
            pattern.append(vector)
        
        y_encoded.append(pattern)
    
    return X_encoded, y_encoded

# let's test it out
# to make it easy to see how pieces fit i've numbered the X & y transforms
# X_1, X_2, y_3, etc...to X_final, y_final

seed(1)

n_samples = 1

n_terms = 3

largest_number = 10

# make pairs

X_1, y_1 = random_sum_pairs(n_samples, n_terms, largest_number)

print('step 1: pairs with sums \n')
print(X_1, y_1)

# convert to strings

X_2, y_2 = to_string(X_1, y_1, n_terms, largest_number)

print('\n step 2: transform to strings \n')
print(X_2, y_2)

# integer encode
# include every character we're using
# even the spaces for padding!

alphabet = ['0','1','2','3','4','5','6','7','8','9','+',' ']

X_3, y_3 = integer_encode(X_2, y_2, alphabet)

print('\n step 3: integer encoding \n')
print(X_3, y_3)

# one hot encode

X_final, y_final = one_hot_encode(X_3, y_3, len(alphabet))

X_array = np.array(X_final)

print('\n final step: one hot encoding \n')
print(X_final, '\n')
print(y_final)
print('\n')
print(X_array.shape)


step 1: pairs with sums 

[[3, 10, 2]] [15]

 step 2: transform to strings 

['  3+10+2'] ['15']

 step 3: integer encoding 

[[11, 11, 3, 10, 1, 0, 10, 2]] [[1, 5]]

 final step: one hot encoding 

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]] 

[[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]]]


(1, 8, 12)


sweet, it works! obvs we're not going to want to do all that for every sample, so let's make a function:

In [216]:

import numpy as np

def make_samples(n_samples, n_numbers, largest_number, alphabet):
    
    # get pairs
    
    X_pairs, y_pairs = random_sum_pairs(n_samples, n_numbers, largest_number)
    
    # pairs to strings
    
    X_strings, y_strings = to_string(X_pairs, y_pairs, n_numbers, largest_number)
    
    # integer encode
    
    X_int, y_int = integer_encode(X_strings, y_strings, alphabet)
    
    # one hot encode
    
    X_encoded, y_encoded = one_hot_encode(X_int, y_int, len(alphabet))
    
    # return as numpy arrays
    
    X, y = np.array(X_encoded), np.array(y_encoded)
    
    return X, y
    


In [217]:
alphabet = [str(x) for x in range(10)] + ['+', ' ']

X_1, y_1 = generate_data(1, 3, 10, alphabet)

print(X_1)
print(X_1.shape)


[[[0 0 0 0 0 0 0 0 0 0 0 1]
  [0 0 0 0 0 0 0 0 0 0 0 1]
  [0 0 0 0 0 0 0 0 0 0 0 1]
  [0 0 0 0 0 1 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 1 0]
  [0 0 1 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 1 0]
  [0 0 0 0 0 0 0 0 1 0 0 0]]]
(1, 8, 12)


### decoding sequences

in order to easily read the results, we'll need to decode them. with one hot encoding, it's easy to use python's `argmax` to get the results: in a matrix of (nearly) all zeros, the highest value will be the `1` denoting a character's category. 

the results can be inverted using `argmax()` to return the index of the category a particular character belongs to, since the `1` denoting it will be the highest number in the otherwise all-zero array.

In [218]:
def decode_results(sequence, alphabet):
    
    int_2_char = dict((i, c) for i, c in enumerate(alphabet))
    
    strings = list()
    
    for seq in sequences:
        
        string_version = int_2_char[argmax(seq)]
        
        strings.append(string_version)
    
    return ''.join(strings)  

### preparing to compile: setting reusable parameters

we'll set a few parameters to the specifics of this problem, then define & compile the model.

#### generating data

we can generate pairs of integer sets and their sums using the functions above; all we need to do is specify how many digits to add together, and the largest number to include.

finally, for encoding we need to give the alphabet--the list of all possible characters the model might encounter.

#### problem-specific model parameters

we need several variables to maximize code reusability, in case we want to adjust various paramters to tweak the model later.

because our __features__ are the possible characters, and because we can choose to use alternate math functions or even padding schemes (thus changing the included characters), we can set this as a variable.

the length of an input sequence can be changed--for this problem we will use terms, which if our highest possible number is 10 and we include 2 mathematical operators (like '+' or '-'), is a total of 8 characters--or, in this problem, __time steps__.

#### a note on the lstm encoder-decoder architecture

the output of an encoder lstm is 2 dimensional--which is a problem since the decoder lstm layer expects a 3 dimensional input. 


to put the data into the right shape to feed directly into the decoder lstm, we'll use a `RepeatVector` layer. encoded input is repeated once per output time step, adding a 3rd dimension and allowing the lstm encoder layer to connect directly to the lstm decoder layer.

the final value to be specified is the number of output time steps to repeat the decoder's output vector--equal to the number of __output time steps__.

because the length of the output sequences can vary (since it's a randomly variable addition problem), we need to set this value to the largest possible number of digits in the answer. in this case, the highest possible sum set is 10+10+10 = 30, which has 2 digits or time steps.

storing this value in a variable will allow for the parameters of the problem to be easily adjusted.


In [219]:
# define & compile

# set number of terms in addition problem

n_terms = 3

# largest number in a term

largest_number = 10

# string versions of all digits, plus symbols used for math & padding
# in an iterable list

alphabet = [str(x) for x in range(10)] + ['+', ' ']

# set number of features
# length of alphabet

n_char_features = len(alphabet)

# length of input sequences
# calculate max possible based on problem parameters

max_in_seq_length = int(n_terms * ceil(log10(largest_number + 1)) + n_terms -1)

# length of output sequences
# calculate maximum possible based on problem parameters

max_out_seq_length = int(ceil(log10(n_terms * (largest_number + 1))))

test to make sure everything's working properly:

In [220]:
print(alphabet)
print('\n')
print(n_char_features)
print('\n')
print(max_in_seq_length)
print('\n')
print(max_out_seq_length)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '+', ' ']


12


8


2


In [221]:
x1, y1 = generate_data(1, n_terms, largest_number, alphabet)

print(x1)
print(x1.shape)
print(y1)

[[[0 0 0 0 0 0 0 0 0 0 0 1]
  [0 0 0 0 0 0 0 0 0 0 0 1]
  [0 0 0 0 0 0 0 0 0 0 0 1]
  [0 0 0 0 0 0 0 0 1 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 1 0]
  [0 0 0 0 0 0 0 0 1 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 1 0]
  [0 0 0 0 0 0 0 1 0 0 0 0]]]
(1, 8, 12)
[[[0 0 1 0 0 0 0 0 0 0 0 0]
  [0 0 0 1 0 0 0 0 0 0 0 0]]]


## define & compile

the input layer of this model will be an lstm layer with 75 memory cells. its unput shape will be the number of samples, the input sequence length, and the number of possible features (characters).

our model uses a `RepeatVector` layer, set to the number of output time steps, to repeat the encoded vector creating a 3d input for the lstm decoder. the decoder will consists of 50 memory cells, and is set to return sequences.

finally, a `TimeDistributed` layer wraps a dense output layer, allowing the layer to repeat without changing its weights for as many output time steps as we need. the dense layer uses a softmax activation, and sorts according to the (in this case, 12) possible output classes. 

the model will be compiled with:

* log __loss__

* adam gradient descent __optimizer__

* accuracy metrics

In [222]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import RepeatVector

# define the model

encoder_decoder_model = Sequential()

# first lstm layer: the encoder

encoder_decoder_model.add(LSTM(75, input_shape=(max_in_seq_length, n_char_features)))

#encoder_decoder_model.add(LSTM(75, input_shape=(8, 12)))


# RepeatVector adapter layer

encoder_decoder_model.add(RepeatVector(max_out_seq_length))

# second lstm layer: decoder

encoder_decoder_model.add(LSTM(50, return_sequences=True))

# TimeDistributed dense layer

encoder_decoder_model.add(TimeDistributed(Dense(n_char_features, activation='softmax')))

# compile the model

encoder_decoder_model.compile(loss='categorical_crossentropy', 
                       optimizer='adam', metrics=['accuracy'])

# check model stats

stats = encoder_decoder_model.summary()

print(stats)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_37 (LSTM)               (None, 75)                26400     
_________________________________________________________________
repeat_vector_19 (RepeatVect (None, 2, 75)             0         
_________________________________________________________________
lstm_38 (LSTM)               (None, 2, 50)             25200     
_________________________________________________________________
time_distributed_19 (TimeDis (None, 2, 12)             612       
Total params: 52,212
Trainable params: 52,212
Non-trainable params: 0
_________________________________________________________________
None


## train/fit

since we can generate as much data as we want whenever we want, there is no need for a train test split.

unlimited data effectively means that we can show the model as many original samples as we want, without the model seeing a sample twice.

in this way, the number of sequences generated is a proxy for the number of epochs. since we're just going to generate as many sequences as possible--rather than recylcing data--the number of epochs is 1.

for this model, we'll use 100,000 training samples, with a batch size of 40.

In [224]:
X, y = generate_data(100000, n_terms, largest_number, alphabet)

encoder_decoder_model.fit(X, y, epochs=1, batch_size=40)

Epoch 1/1
100000/100000 [==============================] - 17s 169us/step - loss: 0.0820 - acc: 0.9868
